## Scraping New York Times articles
Imports

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests as rq
import tqdm

Creating an empty dataframe

In [2]:
df = pd.DataFrame(columns=['title', 'content'])

Function that scraps one article and returns its title and its content

In [3]:
def scrap_article(url):
    try:
        html = rq.get(url).text
        soup = BeautifulSoup(html,'lxml')
        try:
            title = soup.find('span', attrs={'class':'css-rs6kf8'}).get_text().strip()
            content = soup.find('section', attrs={'name': 'articleBody'}).get_text().strip()
            return (title, content)
        except:
            pass
    except:
        pass

Function that returns all articles' links on one section's main page

In [4]:
def get_articles_links(url):
    html = rq.get(url).text
    soup = BeautifulSoup(html, 'lxml')
    titles = soup.find_all('h2', attrs={'class': 'headline'})
    urls  = []
    for el in titles:
        try:
            urls.append(el.find('a')['href'])
        except:
            urls.append(el.parent.parent.get('href'))
    for url in urls:
        df.at[len(df)] = scrap_article(url)

Main function that reads section's links and calls other functions

In [5]:
url = 'https://www.nytimes.com/'
html = rq.get(url).text
soup = BeautifulSoup(html, 'lxml')
sections = soup.find('div', attrs={'data-testid': 'masthead-mini-nav'})
links_sections = sections.find_all('a')
for link in tqdm.tqdm(links_sections):
    get_articles_links(link.get_attribute_list('href')[0])

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [01:13<00:00,  6.60s/it]


Results

In [6]:
df.head()

,title,content
0,‘Yellow Vest’ Protests Shake France. Here’s th...,PARIS — Vincent Picard describes himself as a ...
1,"In a Seething Pro-Brexit Town, Doubts Creep In...","SUNDERLAND, England — They train in a gym near..."
2,Yemen Peace Talks Begin With Agreement to Free...,CAIRO — Yemen’s warring sides agreed on Thursd...
3,‘Already an Exception’: Merkel’s Legacy Is Sha...,"CHEMNITZ, Germany — Two weeks after announcing..."
4,Who Will Replace Angela Merkel as German Conse...,BERLIN — After 18 years under the leadership o...


In [7]:
print('We have {} articles'.format(len(df)))

We have 515 articles


In [8]:
df.to_csv('nyt_articles.csv', index=False)